In [1]:
import tensorflow as tf
import numpy as np
import random
import os
import sys
import tqdm
import csv
from PIL import Image
import cv2
from matplotlib import pyplot as plt

In [2]:
def image_load_and_batch(img_list):
    im = []
    
    for img_name in img_list:
        
        img =cv2.imread(img_name)
        b, g, r = cv2.split(img)
        im.append(cv2.merge([r,g,b]))

            
    return im

In [3]:
def label_make_batch(label_list):
    output = []
    
    for label in label_list:
        x = label[0]
        y = label[1]
        if x == None or y == None:
            output.append(np.zeros((268,480,2)))
            
        else:
            x = x//4
            y = y//4
            ball_map = np.zeros((268,480,2))
            ball_map[x-2:x+2][y-2:y+2] = 1
            output.append(ball_map)
    
    return output
            

In [4]:
def make_batch(all_data):
    
    img_list = []
    label_list = []
    
    keys = list(all_data.keys())
    random.shuffle(keys)
    for key in keys:
        img_list.append(key)
        label_list.append(all_data[key])
    
    train_img = img_list[:8000]
    test_img = img_list[8000:]
    
    train_label = label_list[:8000]
    test_label = label_list[8000:]
    
    return train_img, test_img, train_label, test_label

In [5]:
def Conv_block(data, train=True, filter1=3, filter2=3, maxpool=True, std =1, in_ch=3, out_ch=3):    
    
    
    conv1 = tf.layers.conv2d(data, filters=out_ch,kernel_size=filter1, strides=std, padding="SAME", trainable=train)
    output = tf.layers.conv2d(conv1, filters=out_ch,kernel_size=filter2, strides=1, padding='SAME',trainable=train)
    if maxpool:
        output = tf.layers.max_pooling2d(inputs = output, pool_size=2, strides=2, padding='VALID')
    
    return output

In [6]:
def Upsample_block(data, output_size, kernel=2, std=2):
    
    output = tf.layers.conv2d_transpose(data, filters=output_size, kernel_size=kernel, strides=std, padding="VALID")
    
    return output

In [7]:
def Model(data, label, train, batch_size):

    
    
    with tf.variable_scope("conv1"):
        Conv1_feature = Conv_block(data, train, filter1=7, filter2=3, maxpool=True, std=2,in_ch=3,out_ch=8)
        Conv1_feature = tf.slice(Conv1_feature,[0,0,0,0],[batch_size,268,480,8])

    with tf.variable_scope("conv2"):
        Conv2_feature = Conv_block(Conv1_feature, train, filter1=7, filter2=3, maxpool=True, in_ch=8,out_ch=16)
        
    with tf.variable_scope("conv3"):
        Conv3_feature = Conv_block(Conv2_feature, train, filter1=7, filter2=3, maxpool=True, in_ch=16,out_ch=32)
        
        
     
    
    upsize = Conv1_feature.get_shape().as_list()[-1]      
    with tf.variable_scope("Upsize1"):
        Conv2_up = Upsample_block(Conv2_feature, upsize, kernel=2, std=2)        
    with tf.variable_scope("Upsize2"):
        upsize2 = Conv2_feature.get_shape().as_list()[-1]
        Conv3_up = Upsample_block(Conv3_feature, upsize, kernel=4, std=4)
        '''bias = tf.zeros([batch_size, 2, 480, 8])
        Conv3_up = tf.concat([Conv3_up, bias], axis=1)'''

        
        
    with tf.variable_scope("Concat_conv"):
        Conv_concat = tf.concat([Conv1_feature, Conv2_up, Conv3_up],axis=3)   
    with tf.variable_scope("conv4"):
        Conv4_feature = Conv_block(Conv_concat, train, filter1=7, filter2=3, maxpool=False, in_ch = 56,out_ch=2)
        #print(Conv4_feature)


    return Conv4_feature

In [10]:
def optimizer(data, label, lr=0.0001):
    #print(data)
    logits = tf.nn.softmax(data)
    #print(logits)
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=data, labels=label)
    loss_op = tf.reduce_mean(cross_entropy, name="Loss")
    train_op = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss_op, name="Train_op")
    
    return logits, loss_op, train_op
    

In [11]:
# Data Load Part
# Data Annotate In Deep_Ball_Data/Annote/ and image Deep_Ball_Data/img


#Annotate Data Load
f = open("./Deep_Ball_Data/Annote/cam.csv", 'r', encoding='utf-8')
k = 4
rdr = csv.reader(f)
Data = []
for line in rdr:
    if line[0][0] == 'F':continue
    if line[1] == ' -': 
        Data.append([None, None])
    else:
        x = int(line[1])
        y = int(line[2])
        Data.append([x,y])
                    
f.close()

#Image Data Load

path = "./Deep_Ball_Data/img/graz-arbesbach_5"
img_list = [os.path.join(path , x) for x in os.listdir(path)]


#merge all data Want to shuffle
all_data = {}
for i,img in enumerate(img_list):
    
    all_data[img]= Data[i]

In [ ]:
n_epoch = 10
batch = 0
batch_size = 16
lr = 0.001
total_loss = 0
train=True


tf.reset_default_graph()


input_image = tf.placeholder(tf.float32,[None, 1080, 1920, 3])
label = tf.placeholder(tf.float32,[None, 268, 480, 2])
init = tf.global_variables_initializer()


with tf.Session(config=tf.ConfigProto(gpu_options=(tf.GPUOptions(per_process_gpu_memory_fraction=0.7)), log_device_placement=True)) as sess:
    
    
    output = Model(input_image, label, train, batch_size)
    
    logits, loss, train_op = optimizer(output, label, lr)
    
    
    saver = tf.train.Saver(max_to_keep = 8000000)
    
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    print("Training.. Start..")
    
    for epoch in range(n_epoch):
        counter = 0
        batch = 0
        total_loss = 0
        total_acc = 0
        Train_image, Test_image, Train_label, Test_label = make_batch(all_data)

        while True:
            counter += 1
            train=True


            now_img = image_load_and_batch(Train_image[batch:batch +batch_size])
            now_label = label_make_batch(Train_label[batch:batch +batch_size])

            now_total_pixcel = np.shape(now_label)[0] *np.shape(now_label)[1] *np.shape(now_label)[2]* np.shape(now_label)[3]
           
            logit, losses, _ = sess.run([logits, loss, train_op], feed_dict={input_image:now_img, label:now_label})
            
            
            #print(type(len(np.nonzero(logit - np.array(now_label)))))
            
            now_acc=0
            now_acc= (float(now_total_pixcel) - float(len(np.nonzero(logit - np.array(now_label)[0])))/float(now_total_pixcel)) *100.0
            total_loss += losses
            total_acc += now_acc

            batch = batch+batch_size
           
                
            if batch>3999 or batch>7999:
                
                print("Now...",str(epoch),"... ",str(batch) ,"..End..")
                print("Now Loss..", str(total_loss), " Now Accuracy..", str(total_acc / counter))
                print("Validation Test Start..")
                
                
                test_batch = 0
                total_loss = 0
                total_acc = 0
                train = False
                
                while True:
                    
                    test_img = image_load_and_batch(Test_image[test_batch:test_batch+ batch_size])
                    test_label = label_make_batch(Test_label[test_batch:test_batch+ batch_size])
                    
                    
                    logit, losses, train_op = sess.run([logits, loss, train_op], feed_dict={input_image:test_img, label:test_label})
                    
                    now_acc=0
                    #now_acc= (now_total_pixcel - (len(np.nonzero(logit - now_label))[0])/now_total_pixcel) *100  
                    total_acc += now_acc
                    total_loss += losses
                    test_batch += batch_size
                    
                    if test_batch>1999:
                        
                        print("Validation Test End..")
                        print("Now Loss..", str(total_loss), "Now Accuracy..", str(total_acc / counter))
                        saver.save(sess, './Deep_Ball_Result/Deep_Ball_Model_' + str(epoch)+'_'+str(batch) + '.ckpt')
                        break
                    
                    
                
                
                
            if batch>7999: break
                
            


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Training.. Start..
